# Visibility analysis

The visibility analysis will, at first, be conducted using the "Arealstatistik" (https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik/boden-nutzung-bedeckung-eignung/arealstatistik-schweiz.assetdetail.25885691.html).
At a later stage it might be conducted using a DEM.

In [1]:
# setup
import geopandas as gpd
import pandas as pd
import numpy as np
import os


In [2]:
user_name = 'Fabio'

# create a path to the GeoJSON file
file_path = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand', 'arealstatistik_Grindelwald.csv')

# load data
arealstatistik_GW = pd.read_csv(file_path, sep=';')
print(arealstatistik_GW)
arealstatistik_GW.info()

        E_COORD  N_COORD  AS18_4
0       2620000  1150100       2
1       2620000  1151400       4
2       2620000  1151800       4
3       2620000  1151900       4
4       2620000  1152200       4
...         ...      ...     ...
120696  2629900  1176900       3
120697  2630300  1177400       3
120698  2629000  1161300       4
120699  2629000  1162000       2
120700  2629000  1168000       4

[120701 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120701 entries, 0 to 120700
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   E_COORD  120701 non-null  int64
 1   N_COORD  120701 non-null  int64
 2   AS18_4   120701 non-null  int64
dtypes: int64(3)
memory usage: 2.8 MB


### Set-up

In [3]:
class Point():
    # initialise
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

     # representation
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'


### Main

In [4]:
x = 0
points = []

for _ in range(len(arealstatistik_GW)):
    a = (Point(arealstatistik_GW['E_COORD'].iloc[x], arealstatistik_GW['N_COORD'].iloc[x]))
    points.append(a)
    x += 1

print(f'The first 5 points are: {points[:5]}')
print(f'There are {len(points)} points in the dataset.')

The first 5 points are: [Point(x=2620000, y=1150100), Point(x=2620000, y=1151400), Point(x=2620000, y=1151800), Point(x=2620000, y=1151900), Point(x=2620000, y=1152200)]
There are 120701 points in the dataset.
